In [ ]:
import ivolatility as ivol
import pandas as pd
import os

ivolApiKey = os.getenv('API_KEY')
ivol.setLoginParams(apiKey=ivolApiKey)

# Let's now imagine that we are interested in extracting options on futures from the previous example No.5
getFutData = ivol.setMethod('/futures/eod/futures-info')
futDataRaw = getFutData(date='2023-11-01',underlyingName='E%Mini%S%P%500%')

futuresNeed = getFutData(date='2023-11-01',rootSymbol='ES', MIC = 'XCME')

# Let's now find the prices for these futures for period from '2023-11-01' to '2023-11-10'
geFutPriceData = ivol.setMethod('/futures/single-futures')


finalFutData = pd.DataFrame()
for futSymbol in futuresNeed['FutSymbol']:  # Corrected column name
    futPriceData = geFutPriceData(symbol=futSymbol, from_='2023-11-01', to='2023-11-10')
    finalFutData = pd.concat([finalFutData, futPriceData], axis=0)
    
'''
Creating a list of required futures (take from the previous example) 
with expiration dates between "2023-10-21" and "2024-10-21"
'''

myFutList = finalFutData['Future symbol'][(finalFutData['Future expiration'] > '2023-10-21') & (finalFutData['Future expiration'] < '2024-10-21')].unique()

futOptsList = pd.DataFrame()

'''
Suppose we need call options on ES futures also with expiration dates between "2023-10-21" and "2024-10-21", 
a strike range between 3800 and 3900 and on the date "2023-11-01"
'''

# The process of extracting option tickers based on chosen parameters
getFutOptsData = ivol.setMethod('/futures/eod/option-series-on-date')
for futSymbol in myFutList:
  futOptsNeed = getFutOptsData(date='2023-11-01', futuresSymbol=futSymbol, expFrom='2023-10-21', expTo='2024-10-21', strikeFrom=3800, strikeTo=3900, callPut='C')
  futOptsList = pd.concat([futOptsList, futOptsNeed], axis=0)

finalOptsCalcData = pd.DataFrame()

# Getting futures options pricing data with implied volatilities and Greeks
getFutOptsCalcData = ivol.setMethod('/futures/eod/single-future-option-rawiv')
for futOptSymbol in futOptsList['opt_symbol']:
  futOptsCalcData = getFutOptsCalcData(symbol=futOptSymbol, from_='2023-11-01', to='2023-11-10')
  finalOptsCalcData = pd.concat([finalOptsCalcData, futOptsCalcData], axis=0)

finalOptsCalcData

,date,symbol,option root symbol,Future symbol,exchange,Future expiration,Adjusted close,expiration,strike,Call/Put,...,bid,Settlement price,volume,Open interest,iv,iv_interpolated,vega,delta,gamma,theta
0,2023-11-01,ES,E1C,ES/23Z.CM,CME,2023-12-15,4256.00,2023-11-01,3800.0,C,...,0.00,456.06,8,0,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
0,2023-11-01,ES,E1C,ES/23Z.CM,CME,2023-12-15,4256.00,2023-11-01,3805.0,C,...,0.00,451.06,0,1,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
0,2023-11-01,ES,E1C,ES/23Z.CM,CME,2023-12-15,4256.00,2023-11-01,3810.0,C,...,0.00,446.06,0,0,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
0,2023-11-01,ES,E1C,ES/23Z.CM,CME,2023-12-15,4256.00,2023-11-01,3815.0,C,...,0.00,441.06,0,0,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
0,2023-11-01,ES,E1C,ES/23Z.CM,CME,2023-12-15,4256.00,2023-11-01,3820.0,C,...,0.00,436.06,0,0,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2023-11-06,ES,ES,ES/24U.CM,CME,2024-09-20,4525.50,2024-09-20,3900.0,C,...,680.25,701.25,0,53,0.211864,0.211864,11.233241,0.802354,0.000325,-0.272254
4,2023-11-07,ES,ES,ES/24U.CM,CME,2024-09-20,4538.50,2024-09-20,3900.0,C,...,695.25,713.00,0,53,0.213802,0.213802,11.156722,0.804980,0.000319,-0.272219
5,2023-11-08,ES,ES,ES/24U.CM,CME,2024-09-20,4542.00,2024-09-20,3900.0,C,...,683.50,713.25,0,53,0.211874,0.211874,11.043464,0.807990,0.000319,-0.266070
6,2023-11-09,ES,ES,ES/24U.CM,CME,2024-09-20,4504.75,2024-09-20,3900.0,C,...,656.75,684.25,0,53,0.211750,0.211750,11.318364,0.796780,0.000333,-0.280331
